# 🧱 Base Notebook: Transformers Feature Engineering

Este notebook prepara datasets para Transformer por horizonte (`W12`, `W18`, `W24`) uniendo:
- features base de `3_features` (day0 opcional),
- salidas de clustering (`p_cluster_*`, `confidence`, `entropy`),
- target final.

In [ ]:
# =========================
# 1) Imports + Config
# =========================
from pathlib import Path
import json
import numpy as np
import pandas as pd

try:
    from educational_ai_analytics.config import (
        FEATURES_DATA_DIR, EMBEDDINGS_DATA_DIR, DATA_DIR, W_WINDOWS
    )
except Exception:
    ROOT = Path('/workspace/TFM_education_ai_analytics')
    DATA_DIR = ROOT / 'data'
    FEATURES_DATA_DIR = DATA_DIR / '3_features'
    EMBEDDINGS_DATA_DIR = DATA_DIR / '4_embeddings'
    W_WINDOWS = [12, 18, 24]

OUT_ROOT = DATA_DIR / '6_transformer_features'
HORIZONS = sorted([int(w) for w in W_WINDOWS])
SPLITS = ['training', 'validation', 'test']
INCLUDE_DAY0 = True
SEG_FILE = 'segmentation_gmm_ae.csv'
TARGET_FILE = 'target.csv'

OUT_ROOT.mkdir(parents=True, exist_ok=True)
print('✅ Config cargada')
print('Horizontes:', HORIZONS)
print('Output:', OUT_ROOT)

In [ ]:
# =========================
# 2) Helpers
# =========================
def safe_read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        return pd.DataFrame()
    return pd.read_csv(path, index_col=0).replace([np.inf, -np.inf], np.nan).fillna(0.0)

def load_target(split: str) -> pd.DataFrame:
    tgt = safe_read_csv(FEATURES_DATA_DIR / split / TARGET_FILE)
    if tgt.empty:
        raise FileNotFoundError(f'No target found for split={split}')
    return tgt

def load_day0(split: str) -> pd.DataFrame:
    return safe_read_csv(FEATURES_DATA_DIR / split / 'day0_static_features.csv')

def load_seg(split: str, w: int) -> pd.DataFrame:
    p = EMBEDDINGS_DATA_DIR / split / f'upto_w{w:02d}' / SEG_FILE
    seg = safe_read_csv(p)
    if seg.empty:
        return seg
    cols = [c for c in seg.columns if c.startswith('p_cluster_')]
    extra = [c for c in ['cluster_id', 'confidence', 'entropy', 'entropy_norm'] if c in seg.columns]
    keep = cols + extra
    seg = seg[keep].copy()
    seg.columns = [f'{c}_w{w:02d}' for c in seg.columns]
    return seg

def build_horizon_table(split: str, horizon: int, include_day0: bool = True) -> pd.DataFrame:
    windows = [w for w in HORIZONS if w <= horizon]
    tgt = load_target(split)

    base = tgt[['final_result']].copy()
    if include_day0:
        day0 = load_day0(split)
        if not day0.empty:
            day0 = day0.add_prefix('d0_')
            base = base.join(day0, how='inner')

    for w in windows:
        seg = load_seg(split, w)
        if seg.empty:
            print(f'⚠️ split={split} W{w}: segmentation vacía/no encontrada')
            continue
        base = base.join(seg, how='inner')

    base = base.dropna()
    return base

def export_horizon_dataset(df: pd.DataFrame, split: str, horizon: int):
    out_dir = OUT_ROOT / split / f'w{horizon:02d}'
    out_dir.mkdir(parents=True, exist_ok=True)

    y = df['final_result'].astype(int).values
    feature_df = df.drop(columns=['final_result'])

    prob_cols = [c for c in feature_df.columns if c.startswith('p_cluster_')]
    static_cols = [c for c in feature_df.columns if c not in prob_cols]

    windows = [w for w in HORIZONS if w <= horizon]
    seq_steps = []
    for w in windows:
        step_cols = [c for c in prob_cols if c.endswith(f'_w{w:02d}')]
        if step_cols:
            seq_steps.append(feature_df[step_cols].values.astype(np.float32))

    if len(seq_steps) == 0:
        X_seq = np.zeros((len(feature_df), 1, 1), dtype=np.float32)
    else:
        X_seq = np.stack(seq_steps, axis=1)

    X_static = feature_df[static_cols].values.astype(np.float32) if len(static_cols) else np.zeros((len(feature_df), 0), dtype=np.float32)

    np.save(out_dir / 'X_seq.npy', X_seq)
    np.save(out_dir / 'X_static.npy', X_static)
    np.save(out_dir / 'y.npy', y)

    feature_df.to_csv(out_dir / 'features_flat.csv')
    pd.DataFrame(index=df.index).to_csv(out_dir / 'index.csv')

    meta = {
        'split': split,
        'horizon': int(horizon),
        'n_samples': int(len(df)),
        'seq_shape': list(X_seq.shape),
        'static_shape': list(X_static.shape),
        'windows_used': windows,
        'include_day0': bool(any(c.startswith('d0_') for c in static_cols)),
        'n_prob_features_flat': int(len(prob_cols)),
        'n_static_features': int(len(static_cols))
    }
    (out_dir / 'meta.json').write_text(json.dumps(meta, indent=2, ensure_ascii=False))

    print(f'💾 Exportado {split} W{horizon}: N={len(df)} | seq={X_seq.shape} | static={X_static.shape}')

In [ ]:
# =========================
# 3) Build all datasets
# =========================
for split in SPLITS:
    for horizon in HORIZONS:
        try:
            df_h = build_horizon_table(split=split, horizon=horizon, include_day0=INCLUDE_DAY0)
            if df_h.empty:
                print(f'⚠️ Vacío: split={split}, W={horizon}')
                continue
            export_horizon_dataset(df_h, split=split, horizon=horizon)
        except Exception as e:
            print(f'❌ Error split={split}, W={horizon}: {e}')

In [ ]:
# =========================
# 4) Quick sanity check
# =========================
rows = []
for split in SPLITS:
    for horizon in HORIZONS:
        meta_path = OUT_ROOT / split / f'w{horizon:02d}' / 'meta.json'
        if meta_path.exists():
            m = json.loads(meta_path.read_text())
            rows.append(m)

pd.DataFrame(rows) if rows else print('No se encontraron metadatos exportados.')